In [4]:
# !pip install parse_tweet
import datetime

day = '2022-12-24'
day_to_datetime = datetime.datetime.strptime(day,'%Y-%m-%d')
date = '{0}/{1}'.format(day_to_datetime.month,day_to_datetime.day)
date

ERROR: Could not find a version that satisfies the requirement parse_tweet (from versions: none)
ERROR: No matching distribution found for parse_tweet


'12/24'

In [5]:
import datetime
import re

import pandas as pd

from twitter_text import parse_tweet

def data_cleansing_value(df:pd.DataFrame):
    # 量的変数をクレンジング
    new_df = df.copy()
    new_df['ASucc%'] = ((df['AP'] / df['AA']) * 100).round(1)
    new_df['AEff%'] = (((df['AP']-df['AE']) / df['AA']) * 100).round(1)
    new_df['BASucc%'] = ((df['BAP'] / df['BAA']) * 100).round(1)
    new_df['BAEff%'] = (((df['BAP']-df['BAE']) / df['BAA']) * 100).round(1)
    # AP/S,BP/S: チームでは変更必要
    new_df['AP/S'] = (df['AP'] / df['Set']).round(2)
    new_df['BP/S'] = (df['BP/S'] / df['Set']).round(2)
    new_df['SVEff%'] = ((df['SVP'] * 100 + df['SVx'] * 25 - df['SVE'] * 25) / df['SVA']).round(1)
    new_df['RSucc%'] = ((df['Rx'] * 100 + df['Rg'] * 50) / df['RA']).round(1)
    return new_df


division = input('Select division: ')
if 'm' in division:
    division_j = 'V' + division[1] + '男子'
elif 'w' in division:
    division_j = 'V' + division[1] + '女子'
s_round = '2022-23_regular'
daily_all = pd.read_csv('data/{0}/individual_daily_{1}.csv'.format(division,s_round))
daily_all['Point'] = daily_all['AP'] + daily_all['BP'] + daily_all['SVP']

day = '2022-12-03'
day_to_datetime = datetime.datetime.strptime(day,'%Y-%m-%d')
# ツイート用の日付
date = '{0}/{1}'.format(day_to_datetime.month,day_to_datetime.day)
# 日付指定
today_df = daily_all[daily_all['Date']==day]

# today_df.head()
stats_dict = {
    # カラム名: [0.スタッツ名, 1.条件文, 2.条件・日本語, 3.単位（点や％など）]
    'Point':["総得点", today_df['Set']>=2, "出場数2セット以上","点"],
    'AA':["アタック打数", today_df['Set']>=2, "出場数2セット以上","本"],
    'ASucc%':["アタック決定率", today_df['AA']>=10, "アタック打数10本以上", '%'],
    'BASucc%':["バックアタック決定率", today_df['BAA']>=5, "バックアタック打数5本以上",'%'],
    'BP/S':["ブロック決定本数セット平均", today_df['Set']>=2,"出場数2セット以上","本"],
    'SVEff%':["サーブ効果率", today_df['SVA']>=5, "サーブ打数5本以上",'%'],
    'RSucc%':["サーブレシーブ成功率", today_df['RA']>=5, "サーブレシーブ受数5本以上",'%'],
    }
stats_col_list = ['Point','AA','AP', 'BAP', 'BP', 'SVP', 'SVx', 'RA']
unit_list = ['点','本','%','%','本','点','%','%']
# stats = np.random.choice(stats_list)
for stats_col,stats_info in stats_dict.items():
    message = "【#{0}】本日({1})の成績\n".format(division_j,date)
    ranking_df = today_df.copy()[stats_info[1]].sort_values(by=[stats_col,"Set"],ascending=False).reset_index()
    message += "【{0}】ランキング（{1}）\n".format(stats_info[0],stats_info[2])
    # print(ranking_df[stats_col].head())
    for i in range(len(ranking_df)):
        player = ranking_df.iloc[i,:]['Player']
        stats = ranking_df.iloc[i,:][stats_col]
        pre_stats = ranking_df.iloc[i-1,:][stats_col]
        # 順位
        if stats == pre_stats:
            rank = "    "
        else:
            if i >= 5: break
            rank = "{0}位".format(i+1)
        # rank = "{0}位".format(i+1) if stats != pre_stats else "    "
        message_candidate = "{0} #{1} {2}{3}\n".format(rank,player,stats,stats_info[3])
        # ツイート文字数が超えてたら、追加せずループを抜ける
        if parse_tweet(message+message_candidate).valid == False: break
        message += message_candidate
        # print(parse_tweet(message).valid)
        # if i == 5 and 

    print(message)

ImportError: cannot import name 'parse_tweet' from 'twitter_text' (C:\Users\fuku_k\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\twitter_text\__init__.py)

In [ ]:
# スタッツリーダー(当日)


In [ ]:
# シーズン通算

In [22]:
individual_total = daily_all.groupby(['Team','Player','No.']).sum(numeric_only=True).reset_index()
individual_total = data_cleansing_value(individual_total)
individual_total.head()

,Team,Player,No.,Set,AA,AP,AE,ASucc%,AP/S,BAA,...,SVEff%,RA,Rx,Rg,RSucc%,GameId,Spectators,Point,AEff%,BAEff%
0,クボタスピアーズ,丸一達也,5,5,8,4,1,50.0,0.80,0,...,-3.1,1,0,1,50.0,2367,399,5,37.5,NaN
1,クボタスピアーズ,井上大地,14,18,198,107,13,54.0,5.94,62,...,7.2,0,0,0,NaN,3897,769,118,47.5,54.8
2,クボタスピアーズ,増田宏人,10,18,71,36,4,50.7,2.00,0,...,8.0,12,5,3,54.2,3897,769,50,45.1,NaN
3,クボタスピアーズ,大崎世惇,24,12,6,4,0,66.7,0.33,0,...,14.1,0,0,0,NaN,3897,769,13,66.7,NaN
4,クボタスピアーズ,岡崎誠,4,0,0,0,0,NaN,NaN,0,...,NaN,0,0,0,NaN,793,210,0,NaN,NaN


In [36]:
individual_team[individual_team['Set']>=2].sort_values(by=['Point'],ascending=False)

,Team,Player,No.,Set,AA,AP,AE,ASucc%,AP/S,BAA,...,SVEff%,RA,Rx,Rg,RSucc%,GameId,Spectators,Point,AEff%,BAEff%
145,近畿クラブスフィーダ,山本希望,37,25,111,71,6,64.0,2.84,1,...,6.8,7,4,2,71.4,5451,829,84,58.6,0.0
149,近畿クラブスフィーダ,本多駿,11,23,202,76,20,37.6,3.30,25,...,2.6,38,18,9,59.2,5451,829,80,27.7,32.0
143,近畿クラブスフィーダ,大手一真,34,16,127,52,11,40.9,3.25,18,...,11.1,140,77,32,66.4,3082,338,69,32.3,33.3
154,近畿クラブスフィーダ,谷口晴彦,8,17,77,31,8,40.3,1.82,15,...,7.9,1,1,0,100.0,5451,829,36,29.9,6.7
155,近畿クラブスフィーダ,那須野裕進,13,24,75,29,3,38.7,1.21,0,...,0.7,5,3,2,80.0,5451,829,34,34.7,NaN
146,近畿クラブスフィーダ,山本映樹,1,13,53,22,5,41.5,1.69,1,...,3.8,2,1,1,75.0,3904,691,27,32.1,0.0
144,近畿クラブスフィーダ,小林洸,23,13,54,21,8,38.9,1.62,4,...,5.5,73,30,21,55.5,3122,255,24,24.1,0.0
134,近畿クラブスフィーダ,中村洸稀,2,14,37,16,5,43.2,1.14,6,...,5.0,6,0,2,16.7,4657,455,19,29.7,33.3
151,近畿クラブスフィーダ,永瀬壮人,28,7,23,9,3,39.1,1.29,2,...,0.0,44,23,12,65.9,2329,574,9,26.1,50.0
139,近畿クラブスフィーダ,伴大輝,26,3,11,5,0,45.5,1.67,0,...,9.4,1,1,0,100.0,794,374,7,45.5,NaN


In [6]:
team_daily = pd.read_csv('data/{0}/team_daily_{1}.csv'.format(division,s_round))
team_daily[['Date','Team']]

,Date,Team
0,2022/11/05,奈良ドリーマーズ
1,2022/11/05,千葉ZELVA
2,2022/11/05,トヨタ自動車サンホークス
3,2022/11/05,東京ヴェルディ
4,2022/11/05,クボタスピアーズ
...,...,...
65,2022/12/03,クボタスピアーズ
66,2022/12/03,トヨタ自動車サンホークス
67,2022/12/03,近畿クラブスフィーダ
68,2022/12/03,奈良ドリーマーズ


In [70]:
# team_total
daily_all.groupby(['Team','Date']).sum(numeric_only=True).reset_index()

,Team,Date,Unnamed: 0,No.,Set,AA,AP,AE,ASucc%,AP/S,...,SVE,SVx,SVEff%,RA,Rx,Rg,RSucc%,GameId,Spectators,Point
0,JTサンダーズ広島,2022-10-22,105,130,29,74,40,6,343.5,13.66,...,14,9,23.3,47,31,10,328.3,1470,22260,55
1,JTサンダーズ広島,2022-10-23,105,130,30,82,39,5,292.3,13.66,...,10,15,54.4,52,25,17,347.5,1540,28000,57
2,JTサンダーズ広島,2022-10-29,105,141,32,82,40,3,296.1,13.50,...,19,21,7.0,53,34,6,308.4,1610,12600,51
3,JTサンダーズ広島,2022-10-30,105,130,33,60,33,2,331.0,11.00,...,11,24,21.7,36,22,3,375.7,1680,10920,50
4,JTサンダーズ広島,2022-11-05,105,128,43,103,49,4,327.2,12.50,...,17,25,48.2,77,25,14,273.0,1736,9016,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,東京グレートベアーズ,2022-10-30,105,137,39,110,50,8,291.1,12.83,...,10,17,39.5,78,35,18,400.5,1624,51912,59
76,東京グレートベアーズ,2022-11-05,105,155,44,120,61,4,275.2,12.50,...,20,9,17.2,84,49,17,436.5,1722,26362,70
77,東京グレートベアーズ,2022-11-06,105,155,39,100,47,3,385.6,13.33,...,15,24,106.7,71,28,18,183.1,1792,22722,59
78,東京グレートベアーズ,2022-11-13,105,153,28,80,34,6,361.9,13.34,...,11,16,37.5,60,26,13,189.6,1918,21868,42


In [ ]:
for stats_col in stats_col_list:
    print(stats_col)
    if stats_col == 'Point':
        reg_judge = today_data['Set']>=2
        column = '総得点'
        pattern = ['出場数2セット', 2]
    elif stats_col == 'AA':
        reg_judge = today_data['Set']>=2
        column ='アタック打数'
        pattern = ['出場数2セット', 0]
    elif stats == 'AP':
        reg_judge = today_data['AA']>=10
        column = 'アタック決定率'
        pattern = ['打数10本', 1]
    elif stats == 'BAP':
        reg_judge = today_data['BAA']>=5
        column = 'バックアタック決定率'
        pattern = ['打数5本', 1]
    elif stats == 'BP/S':
        reg_judge = today_data['Set']>=2
        column = 'ブロックセット平均'
        pattern = ['出場数2セット', 0]
    elif stats == 'SVP':
        reg_judge = today_data['SVA']>=5
        column = 'サーブ得点'
        pattern = ['サーブ打数5本', 2]
    elif stats == 'SVx':
        reg_judge = today_data['SVA']>=5
        column = 'サーブ効果率'
        pattern = ['サーブ打数5本', 1]
    else:
        reg_judge = today_data['RA']>=5
        column = 'サーブレシーブ成功率'
        pattern = ['受数5本', 1]
    print(column)

for i in range(len(stats_df)):
        if pattern[1]==1:
            unit = '%'
        elif pattern[1]==2:
            unit = '点'
        else:
            unit = '本'
        
        if i >= 1 and stats_df[column].iloc[i-1] == stats_df[column].iloc[i]:
            tie = '   '
            rank = ''
        else:
            tie = ''
            rank = '{}位'.format(i+1)
        message = '{0}{5} #{1}({2}, {3}{4})\n'.format(rank, stats_df['名前'].iloc[i], stats_df['チーム'].iloc[i], stats_df[column].iloc[i], unit, tie)
        m_len = messages + message
        e_len = len(''.join(p.findall(m_len)))
        tweet_len = len(m_len) - (e_len/2)
        # print(tweet_len)
        if tweet_len <= 140 and stats_df[column].iloc[i] != 0:
            messages += message
        else:
            break
    print(messages)
    if len(stats_df) == 0:
      print('not tweet')
    else: